### Setup

In [5]:
import datetime

In [6]:
import os
pwd = os.path.abspath('.')
os.chdir(os.path.join(pwd, '../src/'))

In [7]:
%load_ext autoreload
%autoreload 2

  ### Get Index data for date range

In [14]:
from markets_insights.datareader import data_reader
import datetime

reader = data_reader.NseIndicesReader()

daterange_reader = data_reader.DateRangeDataReader(reader)

from_date = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = daterange_reader.read(from_date = from_date, to_date = to_date)
print(result.head(5).to_markdown())

### Get daily, monthly and annually aggregrated data
In this example we will use HistoricalDataProcessor class to get data between a date range. HistoricalDataProcessor will also do monthly and annual aggregation of data.

In [13]:
# import classes & setup
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor
from markets_insights.datareader.data_reader import NseIndicesReader
histDataProcessor = HistoricalDataProcessor()

# Fetch and process the data
year_start = datetime.date(2023, 12, 1)
year_end = datetime.date(2023, 12, 31)
result = histDataProcessor.process(NseIndicesReader(), {'from_date': year_start, 'to_date': year_end})

Started to read data
Reading data from 2023-12-01 to 2023-12-29
https://archives.nseindia.com/content/indices/ind_close_all_01122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_04122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_05122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_06122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_07122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_08122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_11122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_12122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_13122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_14122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_15122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_18122023.csv
https://archives.nseindia.com/content/indices/ind_close_all_

In [21]:
from markets_insights.core.column_definition import BaseColumns

print(result.get_monthly_data().sort_values(BaseColumns.Date).head(3).to_markdown())

|      | Identifier                                  | Date                | Month   |      Volume |    Turnover |    Close |     High |        Low |       Open |
|-----:|:--------------------------------------------|:--------------------|:--------|------------:|------------:|---------:|---------:|-----------:|-----------:|
|    0 | INDIA VIX                                   | 2023-01-31 00:00:00 | 2023-01 | 0           | 0           |    16.88 |    19.39 |    11.6425 |    14.8675 |
| 1012 | NIFTY100 ENHANCED ESG                       | 2023-01-31 00:00:00 | 2023-01 | 8.12952e+09 | 4.99243e+12 |  3352.6  |  3490.75 |  3352.6    |  3472.29   |
|  228 | NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30 | 2023-01-31 00:00:00 | 2023-01 | 2.32482e+09 | 1.3303e+12  | 11286.1  | 11599.4  | 11214.5    | 11456.8    |


### Calculation pipeline for RSI

In [30]:
# import classes & setup options
import datetime
from markets_insights.datareader.data_reader import BhavCopyReader
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, MultiDataCalculationPipelines, CalculationPipelineBuilder, HistoricalDataProcessOptions
from markets_insights.calculations.base import DatePartsCalculationWorker

reader = BhavCopyReader()
options = HistoricalDataProcessOptions()
options.include_monthly_data = False
options.include_annual_data = False
histDataProcessor = HistoricalDataProcessor(options)

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(reader, {'from_date': year_start, 'to_date': to_date})

# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline())
histDataProcessor.set_calculation_pipelines(pipelines)

# Run the pipeline
histDataProcessor.run_calculation_pipelines()

Started to read data
HistoricalDataProcessor.get_data took 1 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 18 seconds
HistoricalDataProcessor.process took 19 seconds
RsiCalculationWorker took 9 seconds
ValueCrossedAboveFlagWorker took 1 seconds
ValueCrossedBelowFlagWorker took 1 seconds


In [32]:
from markets_insights.core.column_definition import BaseColumns, CalculatedColumns

print(result.get_daily_data().sort_values(BaseColumns.Date).tail(3)[[BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, CalculatedColumns.RelativeStrengthIndex]].to_markdown())

|        | Identifier   | Date                |   Close |     Rsi |
|-------:|:-------------|:--------------------|--------:|--------:|
| 336063 | RKFORGE      | 2023-12-29 00:00:00 |  725.7  | 46.9257 |
| 329710 | RBL          | 2023-12-29 00:00:00 |  852.95 | 54.8479 |
| 446931 | ZYDUSWELL    | 2023-12-29 00:00:00 | 1681.1  | 72.0492 |


### A real use case: Understand the affect of RSI and Stochastic RSI on price
In this use case, understand the affect of RSI and Stochastic RSI on price

#### Preparing the data
Calculate RSI and Stochastic RSI for each day.
Add a flag for whenever the RSI crosses the control limits (eg: above 75 and below 30)
Calculate the highest and lowest price change in the next 10 trading sessions.

#### Analyse
We will find the average for highest price change and lowest price change whenever the RSI crosses the threshold.

In [33]:
# import classes
from markets_insights.datareader.data_reader import BhavCopyReader

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(BhavCopyReader(), {'from_date': year_start, 'to_date': to_date})

Started to read data
HistoricalDataProcessor.get_data took 1 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 17 seconds
HistoricalDataProcessor.process took 18 seconds


In [34]:
# prepare calculation pipeline
periods = [1, 7, 15, 30, 45]

pipelines = MultiDataCalculationPipelines()
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline(crossing_above_flag_value = 75, crossing_below_flag_value = 30, window = 14))
pipelines.set_item('stoch_rsi', CalculationPipelineBuilder.create_stoch_rsi_calculation_pipeline(crossing_above_flag_value = 80, crossing_below_flag_value = 20, window = 14))
pipelines.set_item('foward_looking_fall', CalculationPipelineBuilder.create_forward_looking_price_fall_pipeline(periods))
pipelines.set_item('foward_looking_rise', CalculationPipelineBuilder.create_forward_looking_price_rise_pipeline(periods))
histDataProcessor.set_calculation_pipelines(pipelines=pipelines)

# run the pipeline and show results
histDataProcessor.run_calculation_pipelines()

daily_data = result.get_daily_data()

# Import constants so its easier to refer to column names
from markets_insights.core.column_definition import BaseColumns, CalculatedColumns

# get names of fwd looking price column names. Since, these column names are auto-generated there no constants for them
fwd_looking_price_fall_cols, fwd_looking_price_rise_cols = [x for x in daily_data.columns if 'HighestPercFallInNext' in x], \
    [x for x in daily_data.columns if 'HighestPercRiseInNext' in x]

RsiCalculationWorker took 9 seconds
ValueCrossedAboveFlagWorker took 1 seconds
ValueCrossedBelowFlagWorker took 1 seconds
StochRsiCalculationWorker took 13 seconds
ValueCrossedAboveFlagWorker took 1 seconds
ValueCrossedAboveFlagWorker took 1 seconds
ValueCrossedBelowFlagWorker took 1 seconds
ValueCrossedBelowFlagWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
LowestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds
HighestPriceInNextNDaysCalculationWorker took 1 seconds


In [35]:
# analyse the median price change % for highest price fall whenever the RSI crosses above
daily_data[
  (daily_data[CalculatedColumns.RsiCrossedAbove])
][fwd_looking_price_fall_cols].median()

HighestPercFallInNext1Days     1.418923
HighestPercFallInNext7Days     3.771446
HighestPercFallInNext15Days    4.777241
HighestPercFallInNext30Days    6.055861
HighestPercFallInNext45Days    6.785467
dtype: float64

In [38]:
# analyse the median price change % for highest price rise whenever the RSI crosses below
daily_data[
  (daily_data[CalculatedColumns.RsiCrossedBelow] == True)
][fwd_looking_price_rise_cols].median()

HighestPercRiseInNext1Days      3.875000
HighestPercRiseInNext7Days      7.589087
HighestPercRiseInNext15Days     9.756772
HighestPercRiseInNext30Days    13.255317
HighestPercRiseInNext45Days    16.292135
dtype: float64

### Performing Arithmetic on Readers

In [4]:
from markets_insights.core.core import InstrumentTypeFilter
from markets_insights.datareader.data_reader import BhavCopyReader, NseDerivatiesReader, NseIndicesReader
from markets_insights.core.core import IdentifierFilter

for_date = datetime.date(2023, 12, 5)

In [9]:
indices_reader = NseIndicesReader()
vix_reader = NseIndicesReader().set_filter(IdentifierFilter("India VIX"))
op_reader = indices_reader / vix_reader
data = op_reader.read(for_date = for_date).query(str(IdentifierFilter("Nifty 50")))

In [35]:
eq_reader = BhavCopyReader()
fut_reader = NseDerivatiesReader().set_filter(InstrumentTypeFilter("FUTSTK"))
op_reader = (fut_reader - eq_reader)
premium_data = op_reader.read(for_date = for_date)

FO-Close - Cash-Close
FO-High - Cash-High
FO-Low - Cash-Low
FO-Open - Cash-Open


In [20]:
from markets_insights.datareader.data_reader import BhavCopyReader
from markets_insights.core.column_definition import BaseColumns
ids = BhavCopyReader().read(for_date=datetime.date(2024, 1, 30))[BaseColumns.Identifier].values
for id in ids:
  print(id)
#ids = map(, lambda x: {x: {"type": "nse_equity"}})

20MICRONS
360ONE
3IINFOLTD
3MINDIA
3PLAND
5PAISA
AARON
AARTECH
AARTIDRUGS
AARTIIND
AARTIPHARM
AARTISURF
AARVEEDEN
AARVI
AAVAS
ABAN
ABB
ABBOTINDIA
ABCAPITAL
ABFRL
ABSLAMC
ABSLBANETF
ABSLLIQUID
ABSLNN50ET
ACC
ACCELYA
ACE
ACEINTEG
ACI
ACL
ACLGATI
ADANIENSOL
ADANIENT
ADANIGREEN
ADANIPORTS
ADFFOODS
ADORWELD
ADSL
ADVANIHOTR
ADVENZYMES
AEGISCHEM
AEROFLEX
AETHER
AFFLE
AGARIND
AGI
AGRITECH
AGROPHOS
AGSTRA
AHL
AHLADA
AHLEAST
AHLUCONT
AIAENG
AIROLAM
AJANTPHARM
AJMERA
AKG
AKSHARCHEM
AKZOINDIA
ALBERTDAVD
ALEMBICLTD
ALICON
ALKALI
ALKEM
ALKYLAMINE
ALLCARGO
ALLSEC
ALOKINDS
ALPA
ALPHAETF
ALPHAGEO
ALPL30IETF
ALPSINDUS
AMBER
AMBICAAGAR
AMBIKCO
AMBUJACEM
AMIORG
AMJLAND
AMNPLST
AMRUTANJAN
ANANDRATHI
ANANTRAJ
ANDHRAPAP
ANDHRSUGAR
ANGELONE
ANMOL
ANUP
ANURAS
APARINDS
APCL
APCOTEXIND
APEX
APLAPOLLO
APLLTD
APOLLOHOSP
APOLLOPIPE
APOLLOTYRE
APOLSINHOT
APTECHT
APTUS
ARCHIES
ARE&M
ARENTERP
ARIES
ARIHANTCAP
ARIHANTSUP
ARMANFIN
AROGRANITE
ARROWGREEN
ARTEMISMED
ARVIND
ARVINDFASN
ARVSMART
ASAHIINDIA
ASAHISONG
ASAL
ASAL

In [19]:
map(ids, lambda x: {x: {"type": "nse_equity"}})

TypeError: 'function' object is not iterable

In [39]:
from markets_insights.core.column_definition import BaseColumns
premium_data['PremiumPerct'] = premium_data[BaseColumns.Close] / premium_data[eq_reader.col_prefix+BaseColumns.Close] * 100

In [41]:
premium_data.sort_values(['PremiumPerct'], ascending=False)

,InstrumentType,Identifier,ExpiryDate,StrkPric,OptionType,FO-PreviousClose,FO-Open,FO-High,FO-Low,FO-Close,...,Cash-Turnover,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13,Close,High,Low,Open,PremiumPerct
348,FUTSTK,DELTACORP,29-Feb-2024,0.0,XX,149.20,0.00,0.00,0.00,149.20,...,7.120425e+08,05-DEC-2023,34060,INE124G01033,NaN,9.55,-141.80,-138.25,-141.50,6.838525
487,FUTSTK,BALRAMCHIN,29-Feb-2024,0.0,XX,484.05,0.00,0.00,0.00,484.05,...,7.126685e+08,05-DEC-2023,28157,INE119A01028,NaN,18.15,-472.15,-461.20,-470.05,3.895686
16,FUTSTK,ZYDUSLIFE,29-Feb-2024,0.0,XX,658.55,0.00,0.00,0.00,658.55,...,4.569366e+08,05-DEC-2023,22038,INE010B01027,NaN,24.50,-643.45,-632.40,-637.90,3.864049
51,FUTSTK,ABBOTINDIA,29-Feb-2024,0.0,XX,24000.00,24000.00,24000.00,24000.00,24000.00,...,7.508119e+08,05-DEC-2023,7742,INE358A01014,NaN,889.55,313.05,999.95,500.00,3.849125
306,FUTSTK,TORNTPHARM,29-Feb-2024,0.0,XX,2180.00,0.00,0.00,0.00,2180.00,...,1.434410e+09,05-DEC-2023,33466,INE685A01028,NaN,80.00,-2160.00,-2096.05,-2135.75,3.809524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,FUTSTK,JUBLFOOD,29-Feb-2024,0.0,XX,547.40,550.00,550.85,537.05,544.80,...,8.654640e+08,05-DEC-2023,33357,INE797F01020,NaN,-12.05,-13.70,-17.95,-11.45,-2.163958
376,FUTSTK,PERSISTENT,25-Jan-2024,0.0,XX,6354.45,6366.05,6378.25,6142.65,6233.35,...,2.249172e+09,05-DEC-2023,51856,INE262H01013,NaN,-146.60,-120.75,-128.15,-103.95,-2.297824
377,FUTSTK,PERSISTENT,29-Feb-2024,0.0,XX,6350.00,6245.25,6245.25,6245.25,6222.90,...,2.249172e+09,05-DEC-2023,51856,INE262H01013,NaN,-157.05,-253.75,-25.55,-224.75,-2.461618
413,FUTSTK,M&MFIN,29-Feb-2024,0.0,XX,273.75,273.00,273.00,259.00,262.95,...,2.533572e+09,05-DEC-2023,60135,INE774D01024,NaN,-7.15,-5.50,-10.50,-5.00,-2.647168


In [21]:
rename_cols = {}
for col in [col for col in premium_data.columns if '_x' in col]:
  rename_cols[col] = f"Cash - {col.replace('_x', '')}"

In [22]:
rename_cols

{'PreviousClose_x': 'Cash - PreviousClose',
 'Open_x': 'Cash - Open',
 'High_x': 'Cash - High',
 'Low_x': 'Cash - Low',
 'Close_x': 'Cash - Close',
 'Volume_x': 'Cash - Volume',
 'Turnover_x': 'Cash - Turnover'}

In [14]:
import pandas as pd
merged_df = pd.merge(cash_data, fut_data, how='inner', on=[BaseColumns.Identifier, BaseColumns.Date], left_index=False, right_index=False)

In [18]:
from markets_insights.core.column_definition import BasePriceColumns
from markets_insights.core.core import TypeHelper

delta = lambda x, y: x - y

for col in TypeHelper.get_class_static_values(BasePriceColumns):
  merged_df[col] = delta(merged_df[col+'_x'], merged_df[col+'_y'])

In [3]:
import datetime
datetime.date.today() - datetime.timedelta(days=1)

datetime.date(2024, 1, 29)

### Extended Data Reader for Nasdaq

In [5]:
import yfinance as yf
import pandas
from markets_insights.datareader.data_reader import DateRangeDataReader
from markets_insights.core.core import Instrumentation
from markets_insights.core.column_definition import BaseColumns

In [19]:
class NasdaqDataReader (DateRangeDataReader):
  def __init__(self, tickers: list = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META', 'TSLA', 'NVDA']):
    super().__init__(reader=None)
    self.tickers = tickers
    self.name = "NasdaqDataReader"

  @Instrumentation.trace(name="NasdaqDataReader.read")
  def read(self, from_date, to_date):
    df_list = list()
    for ticker in self.tickers:
        data = yf.download(ticker, group_by="Ticker", start=from_date, end=to_date)
        data['ticker'] = ticker
        df_list.append(data)

    # combine all dataframes into a single dataframe
    df = pandas.concat(df_list)

    final_data = df.reset_index().rename(columns = self.get_column_name_mappings())
    final_data[BaseColumns.Date] = pandas.to_datetime(final_data[BaseColumns.Date])
    return final_data
  
  def get_column_name_mappings(self):
    return {
      'ticker': BaseColumns.Identifier,
      'OPEN': BaseColumns.Open,
      'HIGH': BaseColumns.High,
      'LOW': BaseColumns.Low,
      'CLOSE': BaseColumns.Close
    }

In [25]:
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
NasdaqDataReader().read(from_date=year_start, to_date=to_date).tail(5)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
NasdaqDataReader.read took 2 seconds


,Date,Open,High,Low,Close,Adj Close,Volume,Identifier
1745,2023-12-22,491.950012,493.829987,484.670013,488.299988,488.299988,25213900,NVDA
1746,2023-12-26,489.679993,496.000000,489.600006,492.790009,492.790009,24420000,NVDA
1747,2023-12-27,495.109985,496.799988,490.850006,494.170013,494.170013,23364800,NVDA
1748,2023-12-28,496.429993,498.839996,494.119995,495.220001,495.220001,24658700,NVDA
1749,2023-12-29,498.130005,499.970001,487.510010,495.220001,495.220001,38869000,NVDA


In [28]:
# import classes & setup options
import datetime
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, MultiDataCalculationPipelines, CalculationPipelineBuilder, HistoricalDataProcessOptions
from markets_insights.calculations.base import DatePartsCalculationWorker

reader = NasdaqDataReader()
options = HistoricalDataProcessOptions()
options.include_monthly_data = False
options.include_annual_data = False
histDataProcessor = HistoricalDataProcessor(options)

# Fetch the data
year_start = datetime.date(2023, 1, 1)
to_date = datetime.date(2023, 12, 31)
result = histDataProcessor.process(reader, {'from_date': year_start, 'to_date': to_date})

# Prepare calculation pipeline
pipelines = MultiDataCalculationPipelines()
pipelines.set_item('date_parts', CalculationPipelineBuilder.create_pipeline_for_worker(DatePartsCalculationWorker()))
pipelines.set_item('rsi', CalculationPipelineBuilder.create_rsi_calculation_pipeline())
histDataProcessor.set_calculation_pipelines(pipelines)

# Run the pipeline
histDataProcessor.run_calculation_pipelines()

Started to read data
Reading data from 2023-01-02 to 2023-12-29
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
NasdaqDataReader.read took 3 seconds
Saving data to file: ../../_data/processed/historical/NasdaqDataReader.csv
get_data took 3 seconds
get_manual_data took 0 seconds
Started basic calculation
add_basic_calc took 0 seconds
process took 3 seconds
DatePartsCalculationWorker took 0 seconds
RsiCalculationWorker took 0 seconds
ValueCrossedAboveFlagWorker took 0 seconds
ValueCrossedBelowFlagWorker took 0 seconds


In [9]:
from markets_insights.core.core import TypeHelper
result.get_daily_data()[TypeHelper.get_class_static_values(BaseColumns)].tail(5)

KeyError: "['PreviousClose', 'Turnover (Rs. Cr.)', 'TOTTRDQTY'] not in index"

In [34]:
from markets_insights.core.column_definition import CalculatedColumns

result.get_daily_data() \
  .sort_values(
    [BaseColumns.Date, BaseColumns.Identifier]
  )[
    [BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, 
     CalculatedColumns.RelativeStrengthIndex]
  ] \
  .tail(7).to_markdown()

'|      | Identifier   | Date                |   Close |     Rsi |\n|-----:|:-------------|:--------------------|--------:|--------:|\n|  248 | AAPL         | 2023-12-28 00:00:00 |  193.58 | 54.4815 |\n|  497 | AMZN         | 2023-12-28 00:00:00 |  153.38 | 63.9387 |\n|  746 | GOOGL        | 2023-12-28 00:00:00 |  140.23 | 61.585  |\n|  995 | META         | 2023-12-28 00:00:00 |  358.32 | 70.2377 |\n| 1244 | MSFT         | 2023-12-28 00:00:00 |  375.28 | 56.909  |\n| 1493 | NVDA         | 2023-12-28 00:00:00 |  495.22 | 58.305  |\n| 1742 | TSLA         | 2023-12-28 00:00:00 |  253.18 | 55.9788 |'

### Create a Calculation Pipeline

In [10]:
import markets_insights

In [39]:
## import modules
from markets_insights.calculations.base import CalculationWorker
from markets_insights.core.core import Instrumentation
from markets_insights.calculations.base import BaseColumns
import pandas

# Implement the worker class. The important aspect here is to override the add_calculated_columns() method
class FibonacciRetracementCalculationWorker (CalculationWorker):
  def __init__(self, time_window: int, level_perct: float):
    self._time_window = time_window
    self._level = level_perct / 100
    self._column_name = 'Fbr' + str(level_perct)

  @Instrumentation.trace(name="FibnocciRetracementCalculationWorker")
  def add_calculated_columns(self, data: pandas.DataFrame):
    identifier_grouped_data: pandas.DataFrame = data.groupby(BaseColumns.Identifier)
    #Since, our dataframe may contain data for multiple symbols, we need to first group them by Identifier
    data[self._column_name] = identifier_grouped_data[BaseColumns.Close].transform(
        lambda x: 
          x.rolling(self._time_window).max() - 
          (
            (x.rolling(self._time_window).max() - x.rolling(self._time_window).min())  * self._level
          )
      )

In [40]:
# Create pipline with the FibnocciRetracementCalculationWorker and run 
from markets_insights.datareader.data_reader import NseIndicesReader
from markets_insights.dataprocess.data_processor import HistoricalDataProcessor, HistoricalDataProcessOptions, \
  MultiDataCalculationPipelines, CalculationPipeline
histDataProcessor = HistoricalDataProcessor(HistoricalDataProcessOptions(include_monthly_data=False, include_annual_data=False))

# Fetch the data
result = histDataProcessor.process(NseIndicesReader(), {'from_date': datetime.date(2023, 12, 1), 'to_date': datetime.date(2023, 12, 31)})

# Prepare calculation pipeline
fbr50_worker = FibonacciRetracementCalculationWorker(time_window=7, level_perct=50)
pipelines = MultiDataCalculationPipelines()
histDataProcessor.set_calculation_pipelines(
  CalculationPipeline(
    workers = [fbr50_worker]
  )
)

# Run the pipeline and get data
histDataProcessor.run_calculation_pipelines()

Started to read data
HistoricalDataProcessor.get_data took 0 seconds
HistoricalDataProcessor.get_manual_data took 0 seconds
HistoricalDataProcessor.run_base_calculations took 0 seconds
HistoricalDataProcessor.process took 0 seconds
FibnocciRetracementCalculationWorker took 0 seconds


In [43]:
## Display the results
print(result.get_daily_data()[[
  BaseColumns.Identifier, BaseColumns.Date, BaseColumns.Close, fbr50_worker._column_name
]].tail(5).to_markdown())

|      | Identifier                    | Date                |    Close |    Fbr50 |
|-----:|:------------------------------|:--------------------|---------:|---------:|
| 2141 | NIFTY COMPOSITE G-SEC INDEX   | 2023-12-29 00:00:00 |  2602.3  |  2599.48 |
| 2142 | NIFTY 10 YR BENCHMARK G-SEC   | 2023-12-29 00:00:00 |  2232.79 |  2230.24 |
| 2143 | NIFTY MIDCAP SELECT           | 2023-12-29 00:00:00 | 10397.5  | 10209.1  |
| 2144 | NIFTY ALPHA LOW-VOLATILITY 30 | 2023-12-29 00:00:00 | 23373.2  | 22886    |
| 2145 | NIFTY50 USD                   | 2023-12-29 00:00:00 |  9048.9  |  8941.77 |
